In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertForMaskedLM
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import string

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2-large').to(device)
t5_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')
t5_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large').to(device)
bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-large-uncased').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
codah = pd.read_csv('/content/drive/MyDrive/DSC253 Group Project/Ashwin_Files/CODAH_Dataset.csv', header=None)
codah.columns = ['type','startphrase', 'ending1', 'ending2', 'ending3', 'ending4','label']
# codah=codah.drop(['drop'],axis=1)
codah['label'] = codah['label'] + 1
codah

,type,startphrase,ending1,ending2,ending3,ending4,label
0,o,I am always very hungry before I go to bed. I am,concerned that this is an illness.,glad that I do not have a kitchen.,fearful that there are monsters under my bed.,tempted to snack when I feel this way.,4
1,o,I am feeling nervous about my midterm tomorrow...,the professor will delay the midterm.,I will doodle on my exam and receive points fo...,my grandpa has diabetes.,I will fail.,4
2,o,My brother is very good at math. He,has a watermelon in his shirt.,is flying out the window.,looks like Billy Bob Buddle.,won a math competition when he was 5.,4
3,o,I feel sickly. I think,I will have a great time.,I will see my brother at the casino.,I will fly out the window.,I will not go to school today.,4
4,o,I am feeling hungry. I think I will,call my mother.,have some ravioli.,fly out the window.,go on a run outside.,2
...,...,...,...,...,...,...,...
2771,i,Sally was on cloud nine after winning the comp...,flew up high in the sky.,was sad.,was very happy.,had to train harder for the next competition.,3
2772,i,John ran like the wind during the race. John,walked during the race.,blew bubbles.,ran slow.,ran fast.,4
2773,i,John hit the nail on the head when he answered...,answered the question exactly right.,answered the question incorrectly.,hammered a nail into the teacher.,build a house using a hammer and nails.,1
2774,i,Diana knew that the project would be a piece o...,knew it would be easy.,loved to eat desserts.,baked pastries for a birthday.,opened a bakery.,1


In [ ]:
#GPT2
def calculate_surprisal_gpt2(sentence_start, sentence_end):
    # Concatenate the start and end phrases
    concatenated_sentence = sentence_start + " " + sentence_end
    # Tokenize the concatenated sentence
    tokens = gpt2_tokenizer.encode(concatenated_sentence, return_tensors='pt').to(device)
    # Tokenize the end phrase separately to determine its length
    end_tokens = gpt2_tokenizer.encode(sentence_end, return_tensors='pt').to(device)
    # Get the logits from the model
    with torch.no_grad():
        outputs = gpt2_model(tokens)
        logits = outputs.logits
    # Calculate the surprisals
    surprisals = []
    decoded_tokens = []
    end_token_start_idx = tokens.size(-1) - end_tokens.size(-1)  # Start index of end phrase tokens
    for i in range(end_token_start_idx, tokens.size(-1)):
        probs = torch.softmax(logits[0, i-1], dim=-1)
        token_id = tokens[0, i]
        surprisal = -torch.log(probs[token_id]).item()
        surprisals.append(surprisal)
        decoded_tokens.append(gpt2_tokenizer.decode(token_id))
    return zip(surprisals, decoded_tokens) if surprisals else None


# # T5 surprisal calculation
def calculate_surprisal_t5(sentence_start, sentence_end):
    #Split the concatenated sentence into words
    words = sentence_start.split() + sentence_end.split()

    start_index = len(sentence_start.split())
    translator = str.maketrans("", "", string.punctuation)

    surprisals = []
    decoded_tokens = []

    #Compute surprisals only on the end phrase
    for i in range(start_index, len(words)):
        # Mask the current token
        word = words[i].translate(translator)
        masked_sentence = ' '.join(words[:i] + ['<extra_id_0>'] + words[i+1:])
        inputs = t5_tokenizer(masked_sentence, return_tensors='pt').to(device)

        with torch.no_grad():
            outputs = t5_model(**inputs, labels=inputs['input_ids'])

        logits = outputs.logits
        mask_token_index = (inputs['input_ids'] == t5_tokenizer.convert_tokens_to_ids('<extra_id_0>')).nonzero(as_tuple=True)[1]
        log_prob = torch.log_softmax(logits[0, mask_token_index, :], dim=-1)

        token_ids = t5_tokenizer.encode(word, add_special_tokens=False)
        surprisal = 0
        if len(token_ids) == 0:
          continue
        if len(token_ids) > 1:
            # If the word is split into multiple tokens, sum the surprisals on each token
            total_surprisal = 0
            for token_id in token_ids:
              total_surprisal+= -log_prob[0, token_id].item()
            surprisal = total_surprisal
        else:
            # If the word is a single token, take the surprisal of that token alone
            token_id = token_ids[0]
            surprisal = -log_prob[0, token_id].item()

        surprisals.append(surprisal)
        decoded_tokens.append(t5_tokenizer.decode(token_ids))

    return list(zip(surprisals, decoded_tokens)) if surprisals else None

def calculate_surprisal_bert(sentence_start, sentence_end):
    # Split the concatenated sentence into words
    words = sentence_start.split() + sentence_end.split()

    start_index = len(sentence_start.split())
    translator = str.maketrans("", "", string.punctuation)

    surprisals = []
    decoded_tokens = []

    # Compute surprisals only on the end phrase
    for i in range(start_index, len(words)):
        # Mask the current token
        word = words[i].translate(translator)
        masked_sentence = ' '.join(words[:i] + [bert_tokenizer.mask_token] + words[i+1:])
        inputs = bert_tokenizer(masked_sentence, return_tensors='pt').to(device)

        with torch.no_grad():
            outputs = bert_model(**inputs)

        logits = outputs.logits
        mask_token_index = (inputs['input_ids'] == bert_tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
        log_prob = torch.log_softmax(logits[0, mask_token_index, :], dim=-1)

        token_ids = bert_tokenizer.encode(word, add_special_tokens=False)
        surprisal = 0
        if len(token_ids) == 0:
          continue
        if len(token_ids) > 1:
            # If the word is split into multiple tokens, sum the surprisals on each token
            total_surprisal = 0
            for token_id in token_ids:
                total_surprisal += -log_prob[0, token_id].item()
            surprisal = total_surprisal
        else:
            # If the word is a single token, take the surprisal of that token alone
            token_id = token_ids[0]
            surprisal = -log_prob[0, token_id].item()

        surprisals.append(surprisal)
        decoded_tokens.append(bert_tokenizer.decode(token_ids))

    return list(zip(surprisals, decoded_tokens)) if surprisals else None

In [ ]:
def calculate_surprisal_result(start_phrase, ending, model_type=''):
    surprisals = None
    if model_type in ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']:
        surprisals = calculate_surprisal_gpt2(start_phrase, ending)
    elif model_type in ['t5', 't5-small', 't5-base', 't5-large']:
        surprisals = calculate_surprisal_t5(start_phrase, ending)
    elif model_type in ['bert', 'bert-base-uncased', 'bert-large-uncased']:
        surprisals = calculate_surprisal_bert(start_phrase, ending)
    else:
        raise ValueError("Unsupported model type")

    if surprisals is None:
        print("Surprisal calculation failed")
        return None

    surprisals = list(surprisals)

    # Find the tuple with the maximum surprisal
    max_surprisal_tuple = max(surprisals, key=lambda x: x[0])

    surprisal_values = [x[0] for x in surprisals]
    avg_surprisal = sum(surprisal_values)/len(surprisal_values)

    return avg_surprisal, max_surprisal_tuple



In [ ]:
def calculate_lowest_surprisal_label(row, model_type=''):
    start_phrase = row['startphrase']
    endings = [row['ending1'], row['ending2'], row['ending3'], row['ending4']]
    surprisals = [calculate_surprisal_result(start_phrase, ending, model_type) for ending in endings]
    avg_surprisals = [x[0] for x in surprisals]
    max_token_surprisals = [x[1] for x in surprisals]
    lowest_average_surprisal_index = avg_surprisals.index(min(avg_surprisals))

    surprisals = [x[0] for x in max_token_surprisals]
    tokens = [x[1] for x in max_token_surprisals]
    lowest_max_token_surprisal_index = surprisals.index(min(surprisals))
    return lowest_average_surprisal_index + 1, lowest_max_token_surprisal_index + 1, tokens

In [ ]:
model = 't5-large'
tqdm.pandas(desc="Progress")
codah[[f'{model}_average_surprisal_prediction', f'{model}_max_word_surprisal_prediction', f'{model}_max_words_each_ending']] = codah.progress_apply(
    lambda row: pd.Series(calculate_lowest_surprisal_label(row, model_type=model)), axis=1
)

Progress: 100%|██████████| 2776/2776 [1:03:43<00:00,  1.38s/it]


In [ ]:
codah

,type,startphrase,ending1,ending2,ending3,ending4,label,t5-large_average_surprisal_prediction,t5-large_max_word_surprisal_prediction,t5-large_max_words_each_ending
0,o,I am always very hungry before I go to bed. I am,concerned that this is an illness.,glad that I do not have a kitchen.,fearful that there are monsters under my bed.,tempted to snack when I feel this way.,4,1,1,"[concerned, a, fearful, tempted]"
1,o,I am feeling nervous about my midterm tomorrow...,the professor will delay the midterm.,I will doodle on my exam and receive points fo...,my grandpa has diabetes.,I will fail.,4,4,4,"[midterm, doodle, grandpa, fail]"
2,o,My brother is very good at math. He,has a watermelon in his shirt.,is flying out the window.,looks like Billy Bob Buddle.,won a math competition when he was 5.,4,4,2,"[watermelon, flying, Buddle, he]"
3,o,I feel sickly. I think,I will have a great time.,I will see my brother at the casino.,I will fly out the window.,I will not go to school today.,4,4,4,"[a, casino, fly, today]"
4,o,I am feeling hungry. I think I will,call my mother.,have some ravioli.,fly out the window.,go on a run outside.,2,1,1,"[call, ravioli, fly, a]"
...,...,...,...,...,...,...,...,...,...,...
2771,i,Sally was on cloud nine after winning the comp...,flew up high in the sky.,was sad.,was very happy.,had to train harder for the next competition.,3,3,3,"[flew, sad, very, train]"
2772,i,John ran like the wind during the race. John,walked during the race.,blew bubbles.,ran slow.,ran fast.,4,4,4,"[walked, bubbles, slow, fast]"
2773,i,John hit the nail on the head when he answered...,answered the question exactly right.,answered the question incorrectly.,hammered a nail into the teacher.,build a house using a hammer and nails.,1,1,1,"[right, incorrectly, hammered, hammer]"
2774,i,Diana knew that the project would be a piece o...,knew it would be easy.,loved to eat desserts.,baked pastries for a birthday.,opened a bakery.,1,1,1,"[it, desserts, pastries, a]"


In [ ]:
codah_df_large = pd.read_csv("/content/drive/MyDrive/DSC253 Group Project/Ashwin_Files/CODAH_Dataset_Large_Models.csv")
columns_to_concat = codah[['t5-large_average_surprisal_prediction', 't5-large_max_word_surprisal_prediction', 't5-large_max_words_each_ending']]
concatenated_df = pd.concat([codah_df_large, columns_to_concat], axis=1)
concatenated_df

,Unnamed: 0,type,startphrase,ending1,ending2,ending3,ending4,label,gpt2-large_average_surprisal_prediction,gpt2-large_max_word_surprisal_prediction,gpt2-large_max_words_each_ending,bert-large-uncased_average_surprisal_prediction,bert-large-uncased_max_word_surprisal_prediction,bert-large-uncased_max_words_each_ending,t5-large_average_surprisal_prediction,t5-large_max_word_surprisal_prediction,t5-large_max_words_each_ending
0,0,o,I am always very hungry before I go to bed. I am,concerned that this is an illness.,glad that I do not have a kitchen.,fearful that there are monsters under my bed.,tempted to snack when I feel this way.,4,2,2,"[' concerned', ' kitchen', ' fearful', ' tempt...",2,3,"['illness', 'kitchen', 'fearful', 'way']",1,1,"[concerned, a, fearful, tempted]"
1,1,o,I am feeling nervous about my midterm tomorrow...,the professor will delay the midterm.,I will doodle on my exam and receive points fo...,my grandpa has diabetes.,I will fail.,4,4,4,"[' delay', 'od', ' grand', ' fail']",2,3,"['midterm', 'doodle', 'diabetes', 'fail']",4,4,"[midterm, doodle, grandpa, fail]"
2,2,o,My brother is very good at math. He,has a watermelon in his shirt.,is flying out the window.,looks like Billy Bob Buddle.,won a math competition when he was 5.,4,4,4,"[' water', ' flying', ' Budd', ' won']",4,2,"['watermelon', 'flying', 'buddle', '5']",4,2,"[watermelon, flying, Buddle, he]"
3,3,o,I feel sickly. I think,I will have a great time.,I will see my brother at the casino.,I will fly out the window.,I will not go to school today.,4,4,4,"[' great', ' casino', ' fly', ' go']",2,2,"['time', 'casino', 'window', 'today']",4,4,"[a, casino, fly, today]"
4,4,o,I am feeling hungry. I think I will,call my mother.,have some ravioli.,fly out the window.,go on a run outside.,2,2,1,"[' call', ' rav', ' fly', ' outside']",1,1,"['mother', 'ravioli', 'window', 'outside']",1,1,"[call, ravioli, fly, a]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771,2771,i,Sally was on cloud nine after winning the comp...,flew up high in the sky.,was sad.,was very happy.,had to train harder for the next competition.,3,3,3,"[' flew', ' sad', ' very', ' train']",4,3,"['sky', 'sad', 'happy', 'competition']",3,3,"[flew, sad, very, train]"
2772,2772,i,John ran like the wind during the race. John,walked during the race.,blew bubbles.,ran slow.,ran fast.,4,4,4,"[' John', ' bubbles', ' slow', ' fast']",1,1,"['walked', 'bubbles', 'slow', 'fast']",4,4,"[walked, bubbles, slow, fast]"
2773,2773,i,John hit the nail on the head when he answered...,answered the question exactly right.,answered the question incorrectly.,hammered a nail into the teacher.,build a house using a hammer and nails.,1,1,1,"[' exactly', ' incorrectly', ' hammered', ' bu...",4,4,"['right', 'incorrectly', 'teacher', 'nails']",1,1,"[right, incorrectly, hammered, hammer]"
2774,2774,i,Diana knew that the project would be a piece o...,knew it would be easy.,loved to eat desserts.,baked pastries for a birthday.,opened a bakery.,1,1,1,"[' Diana', ' desserts', ' baked', ' opened']",1,4,"['easy', 'desserts', 'pastries', 'bakery']",1,1,"[it, desserts, pastries, a]"


In [ ]:
concatenated_df.to_csv("/content/drive/MyDrive/DSC253 Group Project/Ashwin_Files/CODAH_Surprisal_All_Methods_Large_Models.csv")

In [ ]:
model = 'gpt2-large'
tqdm.pandas(desc="Progress")
codah[[f'{model}_average_surprisal_prediction', f'{model}_max_word_surprisal_prediction', f'{model}_max_words_each_ending']] = codah.progress_apply(
    lambda row: pd.Series(calculate_lowest_surprisal_label(row, model_type=model)), axis=1
)

Progress: 100%|██████████| 2776/2776 [02:43<00:00, 16.95it/s]


In [ ]:
model = 'bert-large'
tqdm.pandas(desc="Progress")
codah[[f'{model}_average_surprisal_prediction', f'{model}_max_word_surprisal_prediction', f'{model}_max_words_each_ending']] = codah.progress_apply(
    lambda row: pd.Series(calculate_lowest_surprisal_label(row, model_type=model)), axis=1
)

Progress: 100%|██████████| 2776/2776 [17:37<00:00,  2.62it/s]


In [ ]:
codah

,type,startphrase,ending1,ending2,ending3,ending4,label,bert_average_surprisal_prediction,bert_max_word_surprisal_prediction,bert_max_words_each_ending,gpt2_average_surprisal_prediction,gpt2_max_word_surprisal_prediction,gpt2_max_words_each_ending,t5_average_surprisal_prediction,t5_max_word_surprisal_prediction,t5_max_words_each_ending
0,o,I am always very hungry before I go to bed. I am,concerned that this is an illness.,glad that I do not have a kitchen.,fearful that there are monsters under my bed.,tempted to snack when I feel this way.,4,3,3,"[illness, kitchen, fearful, way]",2,2,"[ concerned, kitchen, fearful, tempted]",1,1,"[concerned, a, fearful, tempted]"
1,o,I am feeling nervous about my midterm tomorrow...,the professor will delay the midterm.,I will doodle on my exam and receive points fo...,my grandpa has diabetes.,I will fail.,4,2,3,"[midterm, doodle, diabetes, fail]",4,4,"[ delay, od, diabetes, fail]",4,4,"[midterm, doodle, grandpa, fail]"
2,o,My brother is very good at math. He,has a watermelon in his shirt.,is flying out the window.,looks like Billy Bob Buddle.,won a math competition when he was 5.,4,4,2,"[watermelon, flying, buddle, 5]",4,4,"[ water, flying, Budd, won]",2,2,"[watermelon, flying, Buddle, a]"
3,o,I feel sickly. I think,I will have a great time.,I will see my brother at the casino.,I will fly out the window.,I will not go to school today.,4,3,3,"[time, casino, window, today]",1,1,"[ great, casino, fly, today]",4,3,"[a, casino, fly, today]"
4,o,I am feeling hungry. I think I will,call my mother.,have some ravioli.,fly out the window.,go on a run outside.,2,3,3,"[mother, ravioli, window, outside]",1,1,"[ call, rav, fly, outside]",3,3,"[call, ravioli, fly, a]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771,i,Sally was on cloud nine after winning the comp...,flew up high in the sky.,was sad.,was very happy.,had to train harder for the next competition.,3,1,1,"[flew, sad, happy, competition]",4,4,"[ Sally, sad, very, train]",3,3,"[flew, sad, happy, harder]"
2772,i,John ran like the wind during the race. John,walked during the race.,blew bubbles.,ran slow.,ran fast.,4,1,1,"[walked, bubbles, slow, fast]",4,4,"[ John, blew, slow, fast]",4,4,"[walked, bubbles, slow, fast]"
2773,i,John hit the nail on the head when he answered...,answered the question exactly right.,answered the question incorrectly.,hammered a nail into the teacher.,build a house using a hammer and nails.,1,4,3,"[right, incorrectly, teacher, nails]",2,2,"[ exactly, incorrectly, hammered, build]",1,1,"[exactly, incorrectly, hammered, hammer]"
2774,i,Diana knew that the project would be a piece o...,knew it would be easy.,loved to eat desserts.,baked pastries for a birthday.,opened a bakery.,1,1,4,"[easy, desserts, pastries, bakery]",1,1,"[ Diana, Diana, baked, opened]",1,1,"[easy, desserts, pastries, a]"


In [ ]:
codah.to_csv('/content/drive/MyDrive/DSC253 Group Project/Ashwin_Files/CODAH_Dataset_Small_Models.csv')